# Modélisations

## Création de la base d'entraînement et de celle de test

In [91]:
import requests
import zipfile
import io
import pandas as pd
from shutil import rmtree
from jyquickhelper import add_notebook_menu
from unidecode import unidecode
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
add_notebook_menu()


In [53]:
insee = pd.read_csv('Données élections et INSEE.csv')
insee = insee.drop("Unnamed: 0", axis = 1)

C:\Users\Xavier\AppData\Local\Temp\ipykernel_12856\3911578995.py:1: DtypeWarning: Columns (1,1557,1558,1559) have mixed types. Specify dtype option on import or set low_memory=False.
  insee = pd.read_csv('Données élections et INSEE.csv')


In [126]:
# On crée la dataframe avec les observations où le maire s'est représenté
insee2 = insee.loc[insee["Tentative de réélection"]=="True"]

In [127]:
# On crée le vecteur contenant résultat
Reelection = insee2["Nom de l'élu en 2020"] == insee2["Nom de l'élu en 2014"]

In [103]:
# On va réduire insee 2 en ne gardant que les données de 2014 et 2020 qui sont des valeurs chiffrées
# Valeurs chiffrées
temp = insee2.select_dtypes(include = ["int64", "float64"])
insee2 = pd.concat([insee2["CODGEO"], temp], axis = 1)
# Années gardées
for i in insee2.columns :
    if i[0:3]!="P19" and i[0:3]!="P13" and i!="CODGEO":
        insee2 = insee2.drop(i, axis = 1)

Si l'on veut être surs que nos modèles soient comparables, il faut centrer et réduire nos données.

In [93]:
temp2 = insee["CODGEO"]

var = insee2.std()
moy = insee2.mean()

insee2 = (insee2-moy)/var
insee2["CODGEO"]=temp2

C:\Users\Xavier\AppData\Local\Temp\ipykernel_12856\544017825.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  var = insee2.std()
C:\Users\Xavier\AppData\Local\Temp\ipykernel_12856\544017825.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  moy = insee2.mean()


In [108]:
# Une partie de nos colonnes contiennent (énormément) de NaNs.
# Seulement quelques colonnes, redondantes entre 2013 et 2020, sont concernées.
# Par simplicité, nous les enlevons donc.
liste = []
for i in insee2.columns:
        if insee2[i].isnull().values.any()==True :
            liste.append(i)
            
insee2 = insee2.drop(liste, axis = 1)        

On crée maintenant les bases d'entraînement et de test pour nos différentes modélisations.

In [111]:
X_train, X_test, y_train, y_test = train_test_split(insee2, Reelection, test_size=0.2, random_state=42)

## 1 - Méthode du lasso

Nous avons ici une base de données avec beaucoup de variables. De ce fait, avec la méthode du lasso, qui ajoute une pénalisation correspondant à la norme 2 du vecteur des $\beta_i$. La formule du lasso est la suivante :
$ \hat{\beta} = argmin_{\beta \in \mathbb{R}^k } = \frac{1}{n} \sum_{i=1}^{n}{(Y_i - X_i^{'}  \beta)^2 + \lambda ||\beta||_1}$ où $\lambda \in \mathbb{R}^{+}$ est un coefficient de pondération.

Ce modèle est continue donc on ne trouvera pas un $\hat{y}_i \in \{ 0; 1\}$ mais dans $[\ 0; 1]\$ correspondant plutôt à une probabilité d'être réélu(e).

In [120]:
clf = linear_model.Lasso(alpha=10)
clf.fit(X_train.drop("CODGEO", axis=1), y_train)
clf

C:\Users\Xavier\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=10)

In [119]:
print(clf.coef_)
print(clf.intercept_)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.